In [30]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from apikey import g_key
gmaps.configure(api_key = g_key)

In [35]:
#Load Part 1 WeatherPy csv into data frame

output_data_file = "/Users/bennetyousuf/Desktop/BCS/python-api-challenge/WeatherPy/cities.csv"

# Read in file and store in Pandas data frame
weather_df = pd.read_csv(output_data_file)

#Drop new column "Unamed:0"
weather_df = weather_df.drop(columns= ["Unnamed: 0"])
weather_df



,City,Lat,Lng,Cloudiness,Max Temp,Humidity,Wind Speed,Country,Date
0,Barrow,71.2906,-156.7887,1,15.80,92,18.41,US,1619382310
1,Mar del Plata,-38.0023,-57.5575,40,64.00,67,11.50,AR,1619382572
2,Rikitea,-23.1203,-134.9692,100,77.86,74,8.10,PF,1619382377
3,Flin Flon,54.7682,-101.8650,90,41.00,36,9.22,CA,1619382573
4,Portland,45.5234,-122.6762,90,59.00,51,5.75,US,1619382425
...,...,...,...,...,...,...,...,...,...
547,Edineţ,48.1681,27.3050,98,45.00,71,5.99,MD,1619382740
548,Mozarlândia,-14.7447,-50.5706,18,82.20,54,6.91,BR,1619382740
549,Yichun,47.7000,128.9000,100,34.14,46,2.37,CN,1619382740
550,Catuday,16.2923,119.8062,5,77.27,74,8.39,PH,1619382566


In [36]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [67]:
#Find mean and max humidity to help with intensity on heatmap image
#Note: mean humidity = 68.01086956521739

max_humidity =weather_df.max()["Humidity"]
max_humidity 

100

In [66]:
# #Humidity heatmap
# Configure gmaps.
# Use the Lat and Lng as locations and Humidity as the weight.
# Add Heatmap layer to map. 


locations = weather_df[["Lat", "Lng"]]

humidity = weather_df["Humidity"].astype(float)

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [48]:
#Create new DataFrame fitting weather criteria
#Filter in weather_df dataframe using .loc to only return values where Max temp is between 70 and 80, wind speed is less than 10mph, and there is zero cloudiness.  
#Drop any rows will null values
#Reset index and then remove unnecessary column "index"
#Display new DataFrame with ideal conditions

ideal_cities_df = weather_df.loc[(weather_df["Max Temp"] > 70) & (weather_df["Max Temp"] < 80) & (weather_df["Cloudiness"] == 0) & (weather_df["Wind Speed"] <10), :]
ideal_cities_df = ideal_cities_df.dropna(how='any')
ideal_cities_df.reset_index(inplace=True)
ideal_cities_df = ideal_cities_df.drop(columns= ["index"])
ideal_cities_df.head()

,City,Lat,Lng,Cloudiness,Max Temp,Humidity,Wind Speed,Country,Date
0,Santiago del Estero,-27.7951,-64.2615,0,74.26,39,6.91,AR,1619382352
1,Quelimane,-17.8786,36.8883,0,77.00,88,5.75,MZ,1619382612
2,Kiryat Gat,31.6100,34.7642,0,73.00,45,3.00,IL,1619382228
3,Arraial do Cabo,-22.9661,-42.0278,0,77.00,78,8.05,BR,1619382432
4,Venado Tuerto,-33.7456,-61.9688,0,71.01,99,8.99,AR,1619382665


In [49]:
# Hotel Map 
#Create and store list of hotels into variable hotel_df

hotel_df = []


for i in range(len(ideal_cities_df)):
    lat = ideal_cities_df.loc[i]['Lat']
    lng = ideal_cities_df.loc[i]['Lng']

#Set parameters to search for hotels within 5000 meters
# Add a "Hotel Name" column to the DataFrame.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
    parameters = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=parameters)
    jsn = requested.json()
    try:
        hotel_df.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")

#Fill in column "Hotel Name" with values from API lookup that were stored in the list for variable hotel_df
#Drop any values in the list from hotel_df that were null
#display new dataframe ideal_cities that now has Hotel Data

ideal_cities_df["Hotel Name"] = hotel_df
ideal_cities_df = ideal_cities_df.dropna(how='any')
ideal_cities_df.head()


,City,Lat,Lng,Cloudiness,Max Temp,Humidity,Wind Speed,Country,Date,Hotel Name
0,Santiago del Estero,-27.7951,-64.2615,0,74.26,39,6.91,AR,1619382352,Santiago del Estero
1,Quelimane,-17.8786,36.8883,0,77.00,88,5.75,MZ,1619382612,Quelimane
2,Kiryat Gat,31.6100,34.7642,0,73.00,45,3.00,IL,1619382228,Kiryat Gat
3,Arraial do Cabo,-22.9661,-42.0278,0,77.00,78,8.05,BR,1619382432,Arraial do Cabo
4,Venado Tuerto,-33.7456,-61.9688,0,71.01,99,8.99,AR,1619382665,Venado Tuerto


In [72]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name 
hotel_info = [info_box_template.format(**row) for index, row in ideal_cities_df.iterrows()]
locations = ideal_cities_df[["Lat", "Lng"]]

In [73]:
# Add marker layer ontop of heat map
#Display figure

fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))